In [1]:
import pandas as pd
import numpy as np
import _pickle

import matplotlib.pyplot as plt

from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_absolute_error

from sklearn.impute import SimpleImputer

In [2]:
SEED=1

In [3]:
with open('model_ready_data', 'rb') as f:
    df = _pickle.load(f)
df.head()

,rent,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
8262,30000,41.043127,28.969064,2,1,100,75,1,8.0,True,...,0,0,1,0,0,0,0,0,0,0
8260,8000,40.979325,28.729905,3,1,145,130,30,5.0,False,...,0,0,1,0,0,0,0,0,0,0
7889,10000,41.057928,28.974291,3,1,110,100,15,5.0,False,...,0,0,1,0,0,0,0,0,0,0
7919,65000,41.152676,28.924586,3,1,165,121,0,3.0,False,...,0,0,0,0,0,1,0,0,0,0
7923,55000,41.207677,29.020296,6,2,450,430,16,4.0,False,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
###################

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10255 entries, 8262 to 10947
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   rent                         10255 non-null  int32  
 1   latitude                     9191 non-null   float64
 2   longitude                    9191 non-null   float64
 3   oda_sayisi                   10255 non-null  int64  
 4   salon_sayisi                 10255 non-null  int64  
 5   brut_m2                      10255 non-null  int64  
 6   net_m2                       10255 non-null  int64  
 7   bina_yasi                    10255 non-null  int32  
 8   kat_sayisi                   10243 non-null  float64
 9   esyali                       10255 non-null  bool   
 10  banyo_sayisi                 10247 non-null  float64
 11  site_icerisinde_binary       10255 non-null  bool   
 12  dogu                         10255 non-null  bool   
 13  bati         

there is no missing value for the categorical variables

In [5]:
X, y = df.drop('rent', axis=1), df['rent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

In [7]:
# [y_train.name] + list(X_train.columns) == df.columns

In [6]:
strategy = 'median'
imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)
train_imputed = pd.DataFrame(imputer.fit_transform(pd.concat([y_train, X_train], axis=1)), 
                             columns=[y_train.name]+list(X_train.columns))
train_imputed

,rent,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
0,12500.0,40.993639,28.873006,3.0,1.0,125.0,110.0,38.0,4.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55000.0,41.067585,29.038525,3.0,1.0,170.0,155.0,50.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15000.0,41.016464,28.589826,1.0,1.0,75.0,70.0,25.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,10000.0,41.048428,28.986134,3.0,1.0,120.0,100.0,15.0,5.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5000.0,41.048428,28.986134,1.0,1.0,60.0,55.0,25.0,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224,12000.0,41.044521,28.977545,1.0,1.0,75.0,65.0,35.0,4.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9225,8500.0,41.048428,28.986134,3.0,1.0,135.0,110.0,20.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9226,12000.0,41.146061,29.041284,2.0,1.0,130.0,125.0,20.0,3.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9227,12500.0,40.921500,29.159100,1.0,1.0,67.0,49.0,6.0,17.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_train_imputed, y_train = train_imputed.drop('rent', axis=1), train_imputed['rent']
X_train_imputed.shape, y_train.shape

((9229, 54), (9229,))

In [15]:
numerical_cols = ['latitude', 'longitude', 'oda_sayisi', 'salon_sayisi', 'brut_m2',
               'net_m2', 'bina_yasi', 'kat_sayisi', 'banyo_sayisi',
               'site_icerisinde_binary', 'cephe_sayisi', 'bulundugu_kat_num']

In [18]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed[numerical_cols]), columns=numerical_cols)
X_train_num_scaled

,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,banyo_sayisi,site_icerisinde_binary,cephe_sayisi,bulundugu_kat_num
0,-0.773885,-0.964285,0.5,0.0,0.238095,0.363636,1.333333,-0.125,0.0,0.0,0.5,0.000
1,0.270573,0.446577,0.5,0.0,0.952381,1.181818,1.904762,-0.125,2.0,0.0,1.5,0.000
2,-0.451480,-3.378078,-0.5,0.0,-0.555556,-0.363636,0.714286,-0.250,0.0,0.0,0.0,-0.700
3,0.000000,0.000000,0.5,0.0,0.158730,0.181818,0.238095,0.000,0.0,0.0,0.0,-0.250
4,0.000000,0.000000,-0.5,0.0,-0.793651,-0.636364,0.714286,-0.375,0.0,0.0,-0.5,-0.575
...,...,...,...,...,...,...,...,...,...,...,...,...
9224,-0.055186,-0.073212,-0.5,0.0,-0.555556,-0.454545,1.190476,-0.125,0.0,0.0,0.0,-0.250
9225,0.000000,0.000000,0.5,0.0,0.396825,0.363636,0.476190,0.125,1.0,0.0,0.5,0.000
9226,1.379014,0.470091,0.0,0.0,0.317460,0.636364,0.476190,-0.250,0.0,0.0,1.5,-0.500
9227,-1.792811,1.474338,-0.5,0.0,-0.682540,-0.745455,-0.190476,1.500,0.0,1.0,0.5,2.000


In [19]:
X_train_scaled = X_train_imputed.copy()
X_train_scaled[numerical_cols] = X_train_num_scaled
X_train_scaled

,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,banyo_sayisi,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
0,-0.773885,-0.964285,0.5,0.0,0.238095,0.363636,1.333333,-0.125,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.270573,0.446577,0.5,0.0,0.952381,1.181818,1.904762,-0.125,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.451480,-3.378078,-0.5,0.0,-0.555556,-0.363636,0.714286,-0.250,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.5,0.0,0.158730,0.181818,0.238095,0.000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,-0.5,0.0,-0.793651,-0.636364,0.714286,-0.375,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224,-0.055186,-0.073212,-0.5,0.0,-0.555556,-0.454545,1.190476,-0.125,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9225,0.000000,0.000000,0.5,0.0,0.396825,0.363636,0.476190,0.125,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9226,1.379014,0.470091,0.0,0.0,0.317460,0.636364,0.476190,-0.250,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9227,-1.792811,1.474338,-0.5,0.0,-0.682540,-0.745455,-0.190476,1.500,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


applying imputation and scaling to test set

In [21]:
X_test 

,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,banyo_sayisi,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
4482,NaN,NaN,2,1,90,80,24,5.0,True,1.0,...,0,0,1,0,0,0,0,0,0,0
5974,41.055509,28.972198,2,1,95,85,11,6.0,False,1.0,...,0,0,0,0,0,0,0,0,0,0
5894,41.078852,28.950906,6,1,260,250,8,5.0,False,2.0,...,0,0,1,0,0,0,0,0,0,0
8056,41.051351,29.005888,2,1,90,85,10,4.0,True,1.0,...,0,0,1,0,0,0,0,0,0,0
6661,40.922518,29.127635,2,1,128,116,31,3.0,True,1.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,NaN,NaN,1,1,75,70,30,4.0,False,1.0,...,0,0,0,0,1,0,0,0,0,0
7665,41.016489,28.681125,3,1,187,170,5,30.0,False,2.0,...,0,0,1,0,0,0,0,0,0,0
3751,40.970407,29.055077,1,1,60,45,0,10.0,False,1.0,...,0,0,0,0,1,0,0,0,0,0
4425,41.027578,28.677838,1,1,85,75,3,4.0,True,1.0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
test_imputed = pd.DataFrame(imputer.transform(pd.concat([y_test, X_test], axis=1)), 
                             columns=[y_test.name]+list(X_test.columns))
test_imputed

,rent,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
0,6000.0,41.048428,28.986134,2.0,1.0,90.0,80.0,24.0,5.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15000.0,41.055509,28.972198,2.0,1.0,95.0,85.0,11.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20000.0,41.078852,28.950906,6.0,1.0,260.0,250.0,8.0,5.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19000.0,41.051351,29.005888,2.0,1.0,90.0,85.0,10.0,4.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12500.0,40.922518,29.127635,2.0,1.0,128.0,116.0,31.0,3.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,7000.0,41.048428,28.986134,1.0,1.0,75.0,70.0,30.0,4.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1022,13000.0,41.016489,28.681125,3.0,1.0,187.0,170.0,5.0,30.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1023,23000.0,40.970407,29.055077,1.0,1.0,60.0,45.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1024,6000.0,41.027578,28.677838,1.0,1.0,85.0,75.0,3.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [23]:
X_test_imputed, y_test = test_imputed.drop('rent', axis=1), test_imputed['rent']
X_test_imputed.shape, y_test.shape

((1026, 54), (1026,))

In [24]:
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_imputed[numerical_cols]), columns=numerical_cols)
X_test_num_scaled

,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,banyo_sayisi,site_icerisinde_binary,cephe_sayisi,bulundugu_kat_num
0,0.000000,0.000000,0.0,0.0,-0.317460,-0.181818,0.666667,0.000,0.0,0.0,0.0,0.250
1,0.100008,-0.118787,0.0,0.0,-0.238095,-0.090909,0.047619,0.125,0.0,0.0,-0.5,0.000
2,0.429726,-0.300280,2.0,0.0,2.380952,2.909091,-0.095238,0.000,1.0,0.0,0.0,0.500
3,0.041276,0.168383,0.0,0.0,-0.317460,-0.090909,0.000000,-0.125,0.0,0.0,0.0,-0.500
4,-1.778426,1.206130,0.0,0.0,0.285714,0.472727,1.000000,-0.250,0.0,0.0,0.5,-0.500
...,...,...,...,...,...,...,...,...,...,...,...,...
1021,0.000000,0.000000,-0.5,0.0,-0.555556,-0.363636,0.952381,-0.125,0.0,0.0,-0.5,-0.575
1022,-0.451135,-2.599852,0.5,0.0,1.222222,1.454545,-0.238095,3.125,1.0,1.0,-0.5,1.250
1023,-1.102021,0.587661,-0.5,0.0,-0.793651,-0.818182,-0.476190,0.625,0.0,0.0,0.5,-0.625
1024,-0.294497,-2.627873,-0.5,0.0,-0.396825,-0.272727,-0.333333,-0.125,0.0,0.0,0.0,-0.700


In [26]:
X_test_scaled = X_test_imputed.copy()
X_test_scaled[numerical_cols] = X_test_num_scaled
X_test_scaled

,latitude,longitude,oda_sayisi,salon_sayisi,brut_m2,net_m2,bina_yasi,kat_sayisi,esyali,banyo_sayisi,...,binned_bulundugu_kat_cat_1,binned_bulundugu_kat_cat_10,binned_bulundugu_kat_cat_11,binned_bulundugu_kat_cat_2,binned_bulundugu_kat_cat_3,binned_bulundugu_kat_cat_4,binned_bulundugu_kat_cat_6,binned_bulundugu_kat_cat_7,binned_bulundugu_kat_cat_8,binned_bulundugu_kat_cat_9
0,0.000000,0.000000,0.0,0.0,-0.317460,-0.181818,0.666667,0.000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.100008,-0.118787,0.0,0.0,-0.238095,-0.090909,0.047619,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.429726,-0.300280,2.0,0.0,2.380952,2.909091,-0.095238,0.000,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.041276,0.168383,0.0,0.0,-0.317460,-0.090909,0.000000,-0.125,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.778426,1.206130,0.0,0.0,0.285714,0.472727,1.000000,-0.250,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,0.000000,0.000000,-0.5,0.0,-0.555556,-0.363636,0.952381,-0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1022,-0.451135,-2.599852,0.5,0.0,1.222222,1.454545,-0.238095,3.125,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1023,-1.102021,0.587661,-0.5,0.0,-0.793651,-0.818182,-0.476190,0.625,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1024,-0.294497,-2.627873,-0.5,0.0,-0.396825,-0.272727,-0.333333,-0.125,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [27]:
model = SVR().fit(X_train_scaled, np.power(y_train, 1/3))

In [28]:
preds_train = np.power(model.predict(X_train_scaled), 3)
mean_absolute_error(y_train, preds_train)

5496.620017077171

In [29]:
preds_test = np.power(model.predict(X_test_scaled), 3)
mean_absolute_error(y_test, preds_test)

5586.829382710604

function form for cross validation

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

def impute_and_scale(X_train, X_val, y_train, y_val, imputation_strategy='median'):
    numerical_cols = ['latitude', 'longitude', 'oda_sayisi', 'salon_sayisi', 'brut_m2',
               'net_m2', 'bina_yasi', 'kat_sayisi', 'banyo_sayisi',
               'site_icerisinde_binary', 'cephe_sayisi', 'bulundugu_kat_num']
    
    # fit the imputer on the train set and impute it
    imputer = SimpleImputer(missing_values=np.nan, strategy=imputation_strategy)
    train_imputed = pd.DataFrame(imputer.fit_transform(pd.concat([y_train, X_train], axis=1)), 
                                 columns=[y_train.name]+list(X_train.columns))
    
    # resplit the train set
    X_train_imputed, y_train = train_imputed.drop('rent', axis=1), train_imputed['rent']
    
    # fit the scaler on the train set and scale it
    scaler = RobustScaler()
    X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed[numerical_cols]), columns=numerical_cols)
    
    X_train_scaled = X_train_imputed.copy()
    X_train_scaled[numerical_cols] = X_train_num_scaled
    
    # impute the val set
    val_imputed = pd.DataFrame(imputer.transform(pd.concat([y_val, X_val], axis=1)), 
                             columns=[y_val.name]+list(X_val.columns))
    
    # resplit the val set
    X_val_imputed, y_val = val_imputed.drop('rent', axis=1), val_imputed['rent']
    
    # scale the val set
    X_val_num_scaled = pd.DataFrame(scaler.transform(X_val_imputed[numerical_cols]), columns=numerical_cols)
    
    X_val_scaled = X_val_imputed.copy()
    X_val_scaled[numerical_cols] = X_val_num_scaled
    
    return X_train_scaled, X_val_scaled

In [6]:
def cross_validation(model, X, y, seed, n_splits=10, imputation_strategy='median'):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cv_scores = []

    for train_idx, val_idx in kf.split(X):
        x_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
        x_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        x_tr, x_val = impute_and_scale(x_tr, x_val, y_tr, y_val, imputation_strategy=imputation_strategy)
        
        model.fit(X=x_tr, y=np.power(y_tr, 1/3))
        pred = np.power(model.predict(x_val), 3)
        cv_scores.append(mean_absolute_error(y_val, pred))
        
    return np.mean(cv_scores)

In [10]:
X, y = df.drop('rent', axis=1), df['rent']

In [8]:
model = SVR()

In [9]:
cross_validation(model, X, y, seed=SEED)

5696.904646514606

# hyperparameter optimization

In [7]:
import optuna

In [8]:
def objective(trial, X, y, seed, n_splits):
    C = trial.suggest_loguniform('C', 0.01, 10)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    degree = 0
    gamma = 'auto'
    epsilon = trial.suggest_loguniform('epsilon', 0.01, 1.0)

    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    elif kernel in ['rbf', 'sigmoid']:
        gamma = trial.suggest_loguniform('gamma', 0.001, 1.0)
    
    params = {'C':C, 'kernel':kernel, 'degree':degree, 
              'gamma':gamma, 'epsilon':epsilon}

    imputation = trial.suggest_categorical('imputation_strategy', ['mean', 'median'])
    
    # train and evaluate the model using the hyperparameters
    #model = SVR(C=C, kernel=kernel, degree=degree, gamma=gamma, epsilon=epsilon)
    model = SVR(**params)
    return cross_validation(model, X, y, seed=SEED, n_splits=n_splits, imputation_strategy=imputation)

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

study = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler())
study.optimize(lambda trial : objective(trial, X=X, y=y, seed=SEED, n_splits=5), n_trials=50)

[I 2023-02-15 21:18:27,201] A new study created in memory with name: no-name-3f3eb849-0aab-49ee-a51c-30e6f04b6b14
[I 2023-02-15 21:18:57,030] Trial 0 finished with value: 7627.475357221648 and parameters: {'C': 2.5778259355760866, 'kernel': 'rbf', 'epsilon': 0.01465192695735506, 'gamma': 0.619806041176648, 'imputation_strategy': 'mean'}. Best is trial 0 with value: 7627.475357221648.
[I 2023-02-15 21:19:21,630] Trial 1 finished with value: 92652067180695.12 and parameters: {'C': 2.0979543836251056, 'kernel': 'poly', 'epsilon': 0.039245591920423636, 'degree': 5, 'imputation_strategy': 'median'}. Best is trial 0 with value: 7627.475357221648.
[I 2023-02-15 21:19:41,611] Trial 2 finished with value: 21110896737.259087 and parameters: {'C': 1.5446016538876401, 'kernel': 'poly', 'epsilon': 0.2393810883689896, 'degree': 4, 'imputation_strategy': 'mean'}. Best is trial 0 with value: 7627.475357221648.
[I 2023-02-15 21:19:56,003] Trial 3 finished with value: 7676.012173334738 and parameters: {

In [12]:
study.best_params

{'C': 6.2409942877045435,
 'kernel': 'rbf',
 'epsilon': 0.2969722590661538,
 'gamma': 0.09611835767838475,
 'imputation_strategy': 'mean'}